In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

In [3]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maddy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\maddy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\maddy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
df = pd.read_csv('Features_For_Traditional_ML_Techniques.csv')

In [5]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\W+', ' ', text)
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    preprocessed_text = ' '.join(lemmatized_tokens)
    return preprocessed_text


In [6]:

df['preprocessed_text'] = df['statement'].apply(preprocess_text)

In [7]:
from nltk.sentiment import SentimentIntensityAnalyzer
from textblob import TextBlob


In [8]:
df['sentiment_score'] = df['preprocessed_text'].apply(lambda x: TextBlob(x).sentiment.polarity)
df['sentiment_label'] = df['sentiment_score'].apply(lambda x: 'Positive' if x > 0 else 'Negative' if x < 0 else 'Neutral')


In [9]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['preprocessed_text'])
df['preprocessed_text'] = tokenizer.texts_to_sequences(df['preprocessed_text'])
vocab_size = len(tokenizer.word_index) + 1

max_len = 100
X = pad_sequences(df['preprocessed_text'], maxlen=max_len)

label_encoder = LabelEncoder()
df['sentiment_label'] = label_encoder.fit_transform(df['sentiment_label'])
y = df['sentiment_label']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [11]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_len))
model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)))
model.add(Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          337600    
                                                                 
 bidirectional (Bidirectiona  (None, 100, 256)         234496    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              164352    
 nal)                                                            
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 6

In [12]:
from tensorflow.keras.callbacks import EarlyStopping

In [13]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3)

In [14]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32, callbacks=[early_stopping])


Epoch 1/10
3355/3355 [==============================] - 1092s 324ms/step - loss: -4114.2568 - accuracy: 0.7599 - val_loss: -11636.0957 - val_accuracy: 0.7703
Epoch 2/10
3355/3355 [==============================] - 1108s 330ms/step - loss: -23520.3770 - accuracy: 0.7628 - val_loss: -38957.9336 - val_accuracy: 0.7609
Epoch 3/10
3355/3355 [==============================] - 1123s 335ms/step - loss: -59220.1523 - accuracy: 0.7631 - val_loss: -83093.1641 - val_accuracy: 0.7694
Epoch 4/10
3355/3355 [==============================] - 1140s 340ms/step - loss: -110410.1172 - accuracy: 0.7666 - val_loss: -142244.1094 - val_accuracy: 0.7677
Epoch 5/10
3355/3355 [==============================] - 1132s 337ms/step - loss: -176877.9375 - accuracy: 0.7720 - val_loss: -217524.6719 - val_accuracy: 0.7727
Epoch 6/10
3355/3355 [==============================] - 1137s 339ms/step - loss: -259286.1562 - accuracy: 0.7722 - val_loss: -309732.4375 - val_accuracy: 0.7735
Epoch 7/10
3355/3355 [===================

In [15]:
# Predict probabilities for the test set
y_pred_prob = model.predict(X_test)

# Convert probabilities to class labels
y_pred = np.argmax(y_pred_prob, axis=1)
y_pred = label_encoder.inverse_transform(y_pred)

# Convert true labels to original class labels
y_true = label_encoder.inverse_transform(y_test)


839/839 [==============================] - 65s 76ms/step


In [16]:
# Print classification report
print(classification_report(y_true, y_pred))

# Print confusion matrix
print(confusion_matrix(y_true, y_pred))

C:\Users\maddy\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\maddy\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

    Negative       0.20      1.00      0.33      5271
     Neutral       0.00      0.00      0.00     15515
    Positive       0.00      0.00      0.00      6054

    accuracy                           0.20     26840
   macro avg       0.07      0.33      0.11     26840
weighted avg       0.04      0.20      0.06     26840

[[ 5271     0     0]
 [15515     0     0]
 [ 6054     0     0]]


C:\Users\maddy\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
model.save('Bi_dir_LSTM_model.h5')